In [ ]:
# Clean and Transform Spotify data
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import time


def cleanPlaylistData(sheetName):

  df = pd.read_excel('SpotifyPlaylistData_20221220.xlsx',sheet_name=sheetName)

  df = df.drop(columns=['Unnamed: 0','artist_uri', 'album_uri'])

  df = df.loc[:,['track_uri', 'track_name', 'artist_name', 'album_name', 'release_date', 'duration_ms', 'explicit', 'popularity']]
  # df['duration_ms'] = pd.to_timedelta(df['duration_ms'], unit='ms')
  
  return df
  
HHPlaylistDF = cleanPlaylistData('HHPlaylist')
CHHPlaylistDF = cleanPlaylistData('CHHPlaylist')


def cleanArtistData(sheetName):

  df = pd.read_excel('SpotifyPlaylistData_20221220.xlsx',sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  
  return df

def cleanAlbumData(sheetName):

  df = pd.read_excel('SpotifyPlaylistData_20221220.xlsx',sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df = df.loc[:,['album_uri', 'artist_uri', 'album_name', 'artist_name', 'release_date', 'label', 'total_tracks', 'popularity']]
  
  return df

HHAlbumsDF = cleanAlbumData('HHAlbums')

def cleanTrackFeaturesData(sheetName):

  df = pd.read_excel('SpotifyPlaylistData_20221220.xlsx',sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df.rename(columns={'uri': 'track_uri'}, inplace=True)
  
  return df

HHTrackFeaturesDF = cleanTrackFeaturesData('HHTrackFeatures')
HHTrackFeaturesDF.head()

In [30]:
# Clean and Transform LastFM data
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import time


def cleanLastFMTrackData(sheetName):

  df = pd.read_excel('LastFMData_20221220.xlsx',sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  # df = df.loc[:,['artist_mbid', 'track_name', 'artist_name', 'listeners', 'playcount']]
  # df = df.sort_values(by='artist_name', ascending=False)
  
  return df

def cleanLastFMArtistData(sheetName):

  df = pd.read_excel('LastFMData_20221220.xlsx',sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  # df = df.loc[:,['artist_mbid', 'track_name', 'artist_name', 'listeners', 'playcount']]
  df = df.drop_duplicates(['artist_name'])
  df = df.reset_index(drop=True)
  # df = df.sort_values(by='artist_name', ascending=False)
  
  return df

def cleanLastFMAlbumData(sheetName):

  df = pd.read_excel('LastFMData_20221220.xlsx',sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  # df = df.loc[:,['artist_mbid', 'track_name', 'artist_name', 'listeners', 'playcount']]
  df = df.drop_duplicates(subset=['album_mbid', 'album_name'])
  df = df.reset_index(drop=True)
  df = df.sort_values(by='listeners', ascending=False)
  
  return df

# HHAlbumData = cleanLastFMAlbumData('HHAlbumData')
# HHAlbumData

# HHTrackData = cleanLastFMTrackData('HHTrackData')
# HHTrackData

HHArtistData = cleanLastFMArtistData('HHArtistData')
HHArtistData

,artist_mbid,artist_name,listeners,playcount
0,c9cd225b-4883-428e-82c2-73e0b6282fb6,Lil Durk,446983,23348973
1,NaN,Juice WRLD,1224367,219018627
2,NaN,A BOOGIE WIT DA HOODIE,698343,38015211
3,NaN,Metro Boomin,727884,35675327
4,b49b81cc-d5b7-4bdd-aadb-385df8de69a6,Drake,4715325,398027860
6,NaN,SoFaygo,225662,14467988
7,NaN,LIL UZI VERT,1197481,144914685
8,NaN,Central Cee,387868,11860607
10,3f334aae-57d4-4787-a048-beec335d47e0,Young Dolph,244425,6381110
12,NaN,Polo G,637656,51949958


In [ ]:
# Push data to excel

import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import datetime as datetime

def openFileToDataFrame(fileName):
    with open(fileName, 'r') as openfile:
        file = json.load(openfile)

    df = pd.DataFrame(file)
    return df

xlxWriter = pd.ExcelWriter('TransformedData.xlsx', engine='xlsxwriter')


xlxWriter.close()